# Convert Coco JSON Annotations to YOLO TXT Files
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [1]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import sys
!{sys.executable} -m pip install pylabel
from pylabel import importer

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# YOLO Format

All the bounding boxes are placed in a `.txt`-file for each image in the dataset.

The order of attributes in each row is ```[class x_centered y_centered width height]```, where all of the values are normalized in the range of [0...1]

The centered values is the center values in both width (x) and height (y) direction, and the width and height tell the size of the bounding box.

An example of such a file:
```
0 0.0172 0.8358 0.0315 0.0347
0 0.1301 0.9474 0.0441 0.0490
0 0.1054 0.9319 0.0936 0.0891
0 0.0275 0.9392 0.0308 0.0311
```


## Import coco annotations 
First we will import annotations from the coco dataset, which are in coco json format. 

In [2]:
#Specify path to the coco.json file
path_to_annotations = "coco_labels_from_labeling_edited.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = "img"

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="labeled_data_coco_format")
dataset.df.head(5)


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,img,898.png,,1,343,343,,,0.001435,0.818474,...,,,,,,1,coralreef,,,1
1,img,898.png,,1,343,343,,,0.108001,0.922881,...,,,,,,1,coralreef,,,1
2,img,898.png,,1,343,343,,,0.058589,0.887334,...,,,,,,1,coralreef,,,1
3,img,898.png,,1,343,343,,,0.012151,0.923665,...,,,,,,1,coralreef,,,1
4,img,1236.png,,2,343,343,,,0.001099,0.819607,...,,,,,,1,coralreef,,,1


## Analyze annotations
Pylabel can calculate basic summary statisticts about the dataset such as the number of files and the classes. 
The dataset is stored as a pandas frame so the developer can do additional exploratory analysis on the dataset. 

In [3]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")


Number of images: 1622
Number of classes: 1
Classes:['coralreef']
Class counts:
coralreef    11698
Name: cat_name, dtype: int64
Path to annotations:
.


## Visualize Annotations 
You can render the bounding boxes for your image to inspect them and confirm that they imported correctly.  

### Fikser sentrering av bokser

In [4]:
data = [[] for _ in range(4)]
for index, row in dataset.df.iterrows():
    xmin = min(row["ann_bbox_xmin"], row["ann_bbox_xmax"])
    ymin = min(row["ann_bbox_ymin"], row["ann_bbox_ymax"])
    data[0].append(xmin + (abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])/2))
    data[1].append(ymin + (abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])/2))
    data[2].append((abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])))
    data[3].append((abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])))

dataset.df["center_x_scaled"] = data[0]
dataset.df["center_y_scaled"] = data[1]
dataset.df["width_scaled"] = data[2]
dataset.df["height_scaled"] = data[3]
dataset.df["center_x_scaled"]

id
0        0.017174
1        0.130056
2        0.105382
3        0.027543
4        0.016957
           ...   
11693    0.723090
11694    0.724826
11695    0.232639
11696    0.032986
11697    0.302951
Name: center_x_scaled, Length: 11698, dtype: float64

In [5]:
img_id = 0
bbox_df = dataset.df.loc[dataset.df['img_filename'] == f'{img_id}.png']
print(bbox_df)

     img_folder img_filename img_path  img_id  img_width  img_height  \
id                                                                     
3375        img        0.png              465        343         343   
3376        img        0.png              465        343         343   
3377        img        0.png              465        343         343   
3378        img        0.png              465        343         343   
3379        img        0.png              465        343         343   
3380        img        0.png              465        343         343   
3381        img        0.png              465        343         343   
3382        img        0.png              465        343         343   
3383        img        0.png              465        343         343   
3384        img        0.png              465        343         343   
3385        img        0.png              465        343         343   
3386        img        0.png              465        343        

In [8]:
import cv2

IMAGE_SIZE = 320
# Set new image size in pandas
dataset.df["img_width"] = IMAGE_SIZE
dataset.df["img_height"] = IMAGE_SIZE

# Open and resize image
img = cv2.imread(f"../images/cloud_cover_lt10/{img_id}.png")
img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_LINEAR)

# Print bounding boxes
for index, row in bbox_df.iterrows():
    x = int(row["center_x_scaled"]*IMAGE_SIZE-row["width_scaled"]*IMAGE_SIZE/2)
    y = int(row["center_y_scaled"]*IMAGE_SIZE-row["height_scaled"]*IMAGE_SIZE/2)
    w = int(row["width_scaled"]*IMAGE_SIZE)
    h = int(row["height_scaled"]*IMAGE_SIZE)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
cv2.imwrite(f"./labeled_{img_id}_320px.png", img)

[5, 131, 37, 39]
[42, 176, 15, 27]
[27, 41, 14, 14]
[1, 1, 45, 31]
[55, 296, 40, 20]
[92, 229, 19, 35]
[47, 62, 19, 19]
[64, 213, 26, 33]
[152, 142, 49, 34]
[69, 79, 17, 18]
[181, 221, 135, 85]
[106, 113, 21, 15]
[134, 127, 24, 16]
[120, 178, 72, 48]


True

In [9]:
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated,center_x_scaled,center_y_scaled,width_scaled,height_scaled
id,,,,,,,,,,,,,,,,,,,,,
0,img,898.png,,1,320,320,,,0.001435,0.818474,...,,1,coralreef,,,1,0.017174,0.835805,0.031477,0.034661
1,img,898.png,,1,320,320,,,0.108001,0.922881,...,,1,coralreef,,,1,0.130056,0.947386,0.044109,0.049011
2,img,898.png,,1,320,320,,,0.058589,0.887334,...,,1,coralreef,,,1,0.105382,0.931860,0.093586,0.089052
3,img,898.png,,1,320,320,,,0.012151,0.923665,...,,1,coralreef,,,1,0.027543,0.939228,0.030785,0.031126
4,img,1236.png,,2,320,320,,,0.001099,0.819607,...,,1,coralreef,,,1,0.016957,0.836500,0.031716,0.033785


# Export to Yolo v5
The PyLabel exporter will export all of the annotations in the dataframe to the desired target format.
Yolo creates one text file for each image in the dataset. 

In [10]:
dataset.export.ExportToYoloV5(output_path="labels/train2022")[0]

'labels/dataset.yaml'